In [150]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Twitter, Kkma
import numpy as np
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, GRU

In [2]:
df1 = pd.read_csv('data/생산_제조.csv',engine='python', index_col=0)
df2 = pd.read_csv('data/IT.csv',engine='python', index_col=0)
df3 = pd.read_csv('data/경영사무.csv',engine='python', index_col=0)
df4 = pd.read_csv('data/무역유통.csv',engine='python', index_col=0)
df5 = pd.read_csv('data/영업_고객상담.csv',engine='python', index_col=0)

In [3]:
df = pd.concat([df1,df2,df3,df4,df5])

In [116]:
df['직무']=df['직무'].replace('생산_제조',1)
df['직무']=df['직무'].replace('IT',2)
df['직무']=df['직무'].replace('경영_사무',3)
df['직무']=df['직무'].replace('무역_유통',4)
df['직무']=df['직무'].replace('영업_고객상담',5)

In [73]:
kkma = Kkma()
df['QuestionNouns1'] = df['질문'].apply(kkma.nouns)
twitter = Twitter()
df['QuestionNouns2'] = df['질문'].apply(twitter.nouns)

C:\Users\smhrd08\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [80]:
df['QuestionNouns3'] = np.nan
for i in range(len(df)):
    df['QuestionNouns3'].iloc[i] = list(set(df['QuestionNouns1'].iloc[i]).intersection(df['QuestionNouns2'].iloc[i]))

C:\Users\smhrd08\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [118]:
X_data = df['QuestionNouns3'].copy()
y_data = df['직무']

In [86]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data) #5572개의 행을 가진 X의 각 행에 토큰화를 수행
sequences = tokenizer.texts_to_sequences(X_data) #단어를 숫자값, 인덱스로 변환하여 저장
print(sequences[:5])

[[5, 1, 89, 390, 30, 8], [113, 136, 16, 11, 58, 25, 1, 127, 18, 53, 152], [9, 5, 26, 436, 4, 244, 96, 20, 107, 64, 228], [5, 14, 604, 1, 89, 2, 30, 8, 13], [10, 11, 108, 1, 4, 97, 19, 84, 21, 2, 75, 41]]


In [87]:
word_to_index = tokenizer.word_index
print(word_to_index)

{'기술': 1, '자': 2, '경험': 3, '본인': 4, '지원': 5, '이내': 6, '무엇': 7, '이유': 8, '직무': 9, '과정': 10, '자신': 11, '동기': 12, '입사': 13, '후': 14, '노력': 15, '생각': 16, '작성': 17, '중': 18, '성장': 19, '서술': 20, '등': 21, '목표': 22, '수': 23, '수행': 24, '선택': 25, '해당': 26, '포부': 27, '관련': 28, '사례': 29, '회사': 30, '역량': 31, '성격': 32, '결과': 33, '점': 34, '설명': 35, '일': 36, '분야': 37, '활동': 38, '극복': 39, '강점': 40, '포함': 41, '달성': 42, '이상': 43, '준비': 44, '가지': 45, '귀하': 46, '생활': 47, '행동': 48, '사항': 49, '관심': 50, '단점': 51, '장단점': 52, '최근': 53, '상황': 54, '때': 55, '계획': 56, '열정': 57, '사회': 58, '업무': 59, '소개': 60, '해결': 61, '역할': 62, '희망': 63, '내용': 64, '중심': 65, '경력': 66, '조직': 67, '인생': 68, '근거': 69, '타인': 70, '문제': 71, '성과': 72, '그것': 73, '가능': 74, '영향': 75, '장점': 76, '가치관': 77, '기억': 78, '계기': 79, '최대': 80, '가치': 81, '진행': 82, '실패': 83, '현재': 84, '최대한': 85, '기여': 86, '성취': 87, '지금': 88, '꿈': 89, '학업': 90, '보완': 91, '도전': 92, '장': 93, '단락': 94, '입력': 95, '능력': 96, '사건': 97, '핵심': 98, '학교생활': 99, '사람': 100, '프로젝트': 101,

In [91]:
print('최대 길이 : %d' % max(len(l) for l in X_data))
print('평균 길이 : %f' % (sum(map(len, X_data))/len(X_data)))

최대 길이 : 63
평균 길이 : 8.510416


In [156]:
X_train, X_test, y_train, y_test = train_test_split(sequences,y_data.tolist(), test_size=0.3, random_state=1234,shuffle=True, stratify= y_data.values)

In [157]:
print(len(X_train),len(y_train))

9912 9912


In [158]:
X_train_seq = sequence.pad_sequences(X_train, maxlen=63)
X_test_seq = sequence.pad_sequences(X_test, maxlen=63)

In [159]:
y_train_ctg = np_utils.to_categorical(y_train)
y_test_ctg = np_utils.to_categorical(y_test)

In [162]:
lstm_model = Sequential()
lstm_model.add(Embedding(2157, 100))
lstm_model.add(LSTM(100, activation='tanh'))
lstm_model.add(Dense(20, activation='relu'))
lstm_model.add(Dense(6, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [163]:
lstm_history = lstm_model.fit(X_train_seq, y_train_ctg, batch_size=64, 
                    epochs=20,validation_data=(X_test_seq, y_test_ctg))

Train on 9912 samples, validate on 4249 samples
Epoch 1/20
9912/9912 [==============================] - 16s 2ms/step - loss: 1.4993 - accuracy: 0.3530 - val_loss: 1.4508 - val_accuracy: 0.3975
Epoch 2/20
9912/9912 [==============================] - 16s 2ms/step - loss: 1.3722 - accuracy: 0.4385 - val_loss: 1.4002 - val_accuracy: 0.4394
Epoch 3/20
9912/9912 [==============================] - 16s 2ms/step - loss: 1.2931 - accuracy: 0.4804 - val_loss: 1.3659 - val_accuracy: 0.4526
Epoch 4/20
9912/9912 [==============================] - 16s 2ms/step - loss: 1.2209 - accuracy: 0.5135 - val_loss: 1.3420 - val_accuracy: 0.4705
Epoch 5/20
9912/9912 [==============================] - 16s 2ms/step - loss: 1.1639 - accuracy: 0.5416 - val_loss: 1.3458 - val_accuracy: 0.4691
Epoch 6/20
9912/9912 [==============================] - 16s 2ms/step - loss: 1.1238 - accuracy: 0.5641 - val_loss: 1.3498 - val_accuracy: 0.4702
Epoch 7/20
9912/9912 [==============================] - 16s 2ms/step - loss: 1.092

In [164]:
gru_model = Sequential()
gru_model.add(Embedding(2157, 100))
gru_model.add(GRU(100, activation='tanh'))
gru_model.add(Dense(20, activation='linear'))
gru_model.add(Dense(6, activation='softmax'))
gru_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [165]:
gru_history = gru_model.fit(X_train_seq, y_train_ctg, batch_size=64, 
                    epochs=20,validation_data=(X_test_seq, y_test_ctg))

Train on 9912 samples, validate on 4249 samples
Epoch 1/20
9912/9912 [==============================] - 18s 2ms/step - loss: 1.4777 - accuracy: 0.3895 - val_loss: 1.4133 - val_accuracy: 0.4156
Epoch 2/20
9912/9912 [==============================] - 17s 2ms/step - loss: 1.3198 - accuracy: 0.4637 - val_loss: 1.3632 - val_accuracy: 0.4429
Epoch 3/20
9912/9912 [==============================] - 17s 2ms/step - loss: 1.2137 - accuracy: 0.5178 - val_loss: 1.3439 - val_accuracy: 0.4688
Epoch 4/20
9912/9912 [==============================] - 17s 2ms/step - loss: 1.1453 - accuracy: 0.5486 - val_loss: 1.3373 - val_accuracy: 0.4695
Epoch 5/20
9912/9912 [==============================] - 17s 2ms/step - loss: 1.0976 - accuracy: 0.5710 - val_loss: 1.3611 - val_accuracy: 0.4792
Epoch 6/20
9912/9912 [==============================] - 17s 2ms/step - loss: 1.0674 - accuracy: 0.5870 - val_loss: 1.3674 - val_accuracy: 0.4818
Epoch 7/20
9912/9912 [==============================] - 17s 2ms/step - loss: 1.037